In [8]:
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
import json
from scipy.stats import norm

key = "key.json"
with open(key, "r") as file:
    api_tokens = json.load(file)

client_secret = api_tokens['client_secret']
client_id = api_tokens['client_id']

client_creds_manager = SpotifyClientCredentials(client_id, client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_creds_manager)

playlist_link = "https://open.spotify.com/playlist/1ltKykGVUUycjkeb55aW53"
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]

playlist_tracks = sp.playlist_tracks(playlist_URI)["items"]

track_sp = sp.audio_features(track_uris)

ids_energy = []

for ids in track_sp:
    ids_energy.append(ids['energy'])

ids_average_energy = sum(ids_energy) / len(ids_energy)


ids_danceability = []

for ids in track_sp:
    ids_danceability.append(ids['danceability'])

ids_average_danceability = sum(ids_danceability) / len(ids_danceability)


ids_acousticness = []

for ids in track_sp:
    ids_acousticness.append(ids['acousticness'])

ids_average_acousticness = sum(ids_acousticness) / len(ids_acousticness)


ids_tempo = []

for ids in track_sp:
    ids_tempo.append(ids['tempo'])

ids_average_tempo = sum(ids_tempo) / len(ids_tempo)

danceability_list = [0.65, 0.15, round(ids_average_danceability, 3)]
energy_list = [0.70, 0.18, round(ids_average_energy, 3)]
acousticness_list = [0.18, 0.22, round(ids_average_acousticness, 3)]
tempo_list = [120.88, 26.9, round(ids_average_tempo, 3)]

attribute_lists = {
    "Danceability": danceability_list,
    "Energy": energy_list,
    "Acousticness": acousticness_list,
    "Tempo": tempo_list
}

def calculate_percentile(mean, sd, value):

    percentile = norm.cdf(value, loc=mean, scale=sd)
    percentile_as_percent = percentile * 100
    return percentile_as_percent

playlist_name = sp.playlist(playlist_URI)
print(f'Playlist Name:', playlist_name['name'])
print("-" * 30)  # Divider for readability 

for attribute, values in attribute_lists.items():
    mean = values[0]
    sd = values[1]
    value = values[2]

    
    percentile = calculate_percentile(mean, sd, value)  
    print(f"This playlist is more obscure than {percentile:.2f}% of playlists in {attribute}")

print("-" * 100)

for i, track in enumerate(playlist_tracks):

    track_name = track["track"]["name"]
    track_pop = track["track"]["popularity"]
    artist_uri = track["track"]["artists"][0]["uri"]
    artist_name = track["track"]["artists"][0]["name"]
    artist_info = sp.artist(artist_uri)
    artist_genres = artist_info["genres"]

    print(f"Track {i + 1}:")
    print(f"  Track Name: {track_name}")
    print(f"  Artist Name: {artist_name}")
    print(f"  Popularity: {track_pop}")
    print(f"  Genres: {', '.join(artist_genres)}")
    print("-" * 30)

Playlist Name: تشريح جثة المؤامرة
------------------------------
This playlist is more obscure than 34.95% of playlists in Danceability
This playlist is more obscure than 31.44% of playlists in Energy
This playlist is more obscure than 76.65% of playlists in Acousticness
This playlist is more obscure than 33.53% of playlists in Tempo
----------------------------------------------------------------------------------------------------
Track 1:
  Track Name: Autopsie d'un complot
  Artist Name: Ahmed Malek
  Popularity: 48
  Genres: arab groove
------------------------------
Track 2:
  Track Name: Yarait
  Artist Name: Ahmed Ben Ali
  Popularity: 26
  Genres: libyan pop, reggae maghreb
------------------------------
Track 3:
  Track Name: Only World
  Artist Name: Ibrahim Hesnawi
  Popularity: 28
  Genres: arab groove
------------------------------
Track 4:
  Track Name: Addayer
  Artist Name: Adib Abou Antoun
  Popularity: 22
  Genres: 
------------------------------
Track 5:
  Track Nam